# Simplification du réseau routier

Mon réseau de route me sera de support pour calculer mes indicateurs de proximité le long du réseau piéton.
Maintenant que j'ai telecharger le réseau, une des choses qu'il faut s'assurer c'est que les intersections sont bien 

In [16]:
%reset -f

In [17]:
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
from shapely.ops import split, unary_union, snap
from rtree import index 

import geopandas as gpd
from shapely.geometry import LineString, MultiLineString, Point, MultiPoint
from shapely.ops import split, unary_union, snap
from rtree import index as rtree_index  # Utilisation de rtree pour l'index spatial

import warnings
warnings.filterwarnings('ignore')

In [18]:
roads = gpd.read_file("../../Proximity/data/processed/roads_network/osm_foot_network_Marseille_2025-10-01.gpkg")
roads.head()

,osm_id,highway,name,access,foot,surface,incline,smoothness,width,sidewalk,crossing,lit,bridge,tunnel,layer,oneway,geometry
0,2682531,service,,,,,,,,,,,,,0,no,"LINESTRING (913982.648 6247541.048, 914032.152..."
1,2699768,path,,,,,,,,,,,,,0,no,"LINESTRING (909518.351 6239066.723, 909533.383..."
2,2699769,path,,,,,,,,,,,,,0,no,"LINESTRING (909448.07 6239122.435, 909435.263 ..."
3,3617343,primary,,,,asphalt,,,,,,,,,0,no,"LINESTRING (910535.741 6250520.857, 910540.587..."
4,3617344,primary,,,,,,,,,,,,,0,no,"LINESTRING (910571.976 6250128.531, 910573.41 ..."


In [19]:
# Je verifie le type de géométrie 
print(roads.geometry.type.unique())

['LineString']


## Préparation et extraction des LineString

In [20]:
# Geom valides
def validate_geometry(geom):
    try:
        if geom is None or geom.is_empty or not geom.is_valid:
            return None
        return geom
    except Exception as e:
        print(f"Erreur lors de la validation de la géométrie : {e}")
        return None

In [21]:
# Je supprime les geométrie non valides
roads['geometry'] = roads['geometry'].apply(validate_geometry)
roads = roads.dropna(subset=['geometry'])
print(f"Nombre de géométries valides : {len(roads)}")

# J'enleve les tunnels (Naive !)
# roads = roads[roads['tunnel'] != 'T'] 
# print(f"Lignes après exclusion des tunnels : {len(roads)}")

Nombre de géométries valides : 96901


In [22]:
# Extraction de tous les segments (LineString)
lines = []      # Liste des géométries de type LineString
line_ids = []   # Référence (ici osm_id) à la ligne d'origine

for idx, row in roads.iterrows():
    geom = row.geometry
    if geom.geom_type == 'LineString':
        lines.append(geom)
        line_ids.append(row.osm_id)
    elif geom.geom_type == 'MultiLineString':
        for sub_geom in geom.geoms:
            lines.append(sub_geom)
            line_ids.append(row.osm_id)

print(f"Nombre total de segments extraits : {len(lines)}")

Nombre total de segments extraits : 96901


In [23]:
print(roads.geometry.type.unique())

['LineString']


## Calcul des points d'intersection entre segments

In [24]:
# Construction d'un index spatial pour accélérer la recherche
line_index = rtree_index.Index()
for pos, line in enumerate(lines):
    line_index.insert(pos, line.bounds)

In [25]:
intersection_points = []  # Liste pour stocker les intersections

# Pour chaque paire de lignes (en utilisant l'index), je calcul l'intersection
for i, line in enumerate(lines):
    possible_indexes = list(line_index.intersection(line.bounds))
    for j in possible_indexes:
        if i >= j:
            continue  # pour éviter les redondances (i=j ou déjà traité)
        other_line = lines[j]
        inter = line.intersection(other_line)
        if inter.is_empty:
            continue
        # Si l'intersection est un Point
        if inter.geom_type == 'Point':
            intersection_points.append(inter)
        # Si l'intersection renvoie plusieurs points
        elif inter.geom_type == 'MultiPoint':
            intersection_points.extend(list(inter.geoms))
        # Si l'intersection est un LineString ou MultiLineString (cas de recouvrement)
        elif inter.geom_type in ['LineString', 'MultiLineString']:
            if inter.geom_type == 'LineString':
                intersection_points.append(Point(inter.coords[0]))
                intersection_points.append(Point(inter.coords[-1]))
            else:
                for sub in inter.geoms:
                    intersection_points.append(Point(sub.coords[0]))
                    intersection_points.append(Point(sub.coords[-1]))

print(f"Nombre de points d'intersection bruts : {len(intersection_points)}")

Nombre de points d'intersection bruts : 160997


In [26]:
# Regrouper les points pour éliminer d'éventuels doublons
if intersection_points:
    intersection_union = unary_union(intersection_points)
    if intersection_union.geom_type == 'Point':
        intersection_union = MultiPoint([intersection_union])
else:
    intersection_union = None

# Construction de l'index spatial pour les points d'intersection

In [27]:
# (pour limiter le nombre de points à considérer par ligne)
if intersection_union is not None:
    if intersection_union.geom_type == 'MultiPoint':
        pts = list(intersection_union.geoms)
    elif intersection_union.geom_type == 'Point':
        pts = [intersection_union]
    else:
        pts = []
else:
    pts = []

In [28]:
pt_index = rtree_index.Index()
for i, pt in enumerate(pts):
    pt_index.insert(i, pt.bounds)

In [29]:
def get_local_intersections(line, buffer=0.001):
    """
    Retourne l'union (MultiPoint) des points d'intersection se trouvant à
    moins de 'buffer' de la ligne.
    """
    minx, miny, maxx, maxy = line.bounds
    # Élargir légèrement l'enveloppe
    bbox = (minx - buffer, miny - buffer, maxx + buffer, maxy + buffer)
    candidate_ids = list(pt_index.intersection(bbox))
    selected_points = [pts[i] for i in candidate_ids if line.distance(pts[i]) < buffer]
    if selected_points:
        union_local = unary_union(selected_points)
        if union_local.geom_type == 'Point':
            union_local = MultiPoint([union_local])
        return union_local
    else:
        return None

## Découpage des lignes aux intersections (nœudisation)

In [30]:
tolerance = 1e-7  # Tolérance pour le snapping

# Boucle séquentielle 
final_segments = []  # Liste des segments finaux (LineString)
final_osm_ids = []   # Référence d'origine

for osm_id, line in zip(line_ids, lines):
    local_intersections = get_local_intersections(line, buffer=0.001)
    if local_intersections is not None:
        # Effectuer un snapping de la ligne sur les points locaux
        snapped_line = snap(line, local_intersections, tolerance)
        try:
            split_result = split(snapped_line, local_intersections)
        except Exception as e:
            print(f"Erreur lors du split de la ligne (osm_id={osm_id}) : {e}")
            continue
        for segment in split_result.geoms:
            if segment.length > 0 and segment.geom_type == 'LineString':
                final_segments.append(segment)
                final_osm_ids.append(osm_id)
    else:
        final_segments.append(line)
        final_osm_ids.append(osm_id)

print(f"Nombre total de segments après découpage (séquentiel) : {len(final_segments)}")

Nombre total de segments après découpage (séquentiel) : 193614


## Création du GeoDataFrame final contenant uniquement des LineString

In [31]:
final_network_gdf = gpd.GeoDataFrame({'osm_id': final_osm_ids},
                                     geometry=final_segments,
                                     crs=roads.crs)

print(f"Réseau final (uniquement des LineString) : {len(final_network_gdf)}")

Réseau final (uniquement des LineString) : 193614


## Simplifier les données pour sauvegarde

Je ne garde que fid et la géometries c'est ce dont j'ai besoin pour la suite

In [32]:
geo_path = "../../Proximity/data/processed/roads_network/osm_foot_network_Marseille_simplified_2025-10-01.gpkg"
final_network_gdf = final_network_gdf[['geometry']]
print(final_network_gdf.head())
final_network_gdf = final_network_gdf.to_crs("EPSG:2154")
final_network_gdf.to_file(geo_path, driver='GPKG', layer='edges')
print(f"Fichier sauvegardé avec succès : {geo_path}")

                                            geometry
0  LINESTRING (913982.648 6247541.048, 914032.152...
1  LINESTRING (909518.351 6239066.723, 909533.383...
2  LINESTRING (909448.07 6239122.435, 909435.263 ...
3  LINESTRING (910535.741 6250520.857, 910540.587...
4  LINESTRING (910557.379 6250589.626, 910560.121...
Fichier sauvegardé avec succès : ../../Proximity/data/processed/roads_network/osm_foot_network_Marseille_simplified_2025-10-01.gpkg
